<a href="https://colab.research.google.com/github/cosimo-schiavoni/Massive_Data_Project/blob/main/Downloads_TFOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [1]:
!wget -q https://www-eu.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz
!rm spark-3.2.1-bin-hadoop2.7.tgz
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install findspark
!pip install pyspark
!pip install -q kaggle
!pip install tensorflowonspark
!pip install sparkdl
!pip install tensorframes
!pip install petastorm

     |████████████████████████████████| 281.4 MB 29 kB/s 
     |████████████████████████████████| 198 kB 23.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3bfbc5851f87a69c85836735b0b8a5550ece6796e35eda58f5f01d170988318e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 45 kB 1.8 MB/s 
     |████████████████████████████████| 99 kB 3.5 MB/s 
     |████████████████████████████████| 284 kB 4.0 MB/s 
     |████████████████████████████████| 44 kB 2.6 MB/s 
     |████████████████████████████████| 136 kB 53.7 MB/s 


In [3]:
# def main():
def main_fun(argv, ctx):
  import tensorflow as tf

if __name__ == '__main__':
  # tf.app.run()
  #Spark
  import pyspark
  from pyspark.sql import SparkSession
  from pyspark import SparkContext, SparkConf
  from pyspark.sql import SQLContext
  from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, VectorAssembler
  from pyspark.ml import Pipeline
  from pyspark.sql.functions import rand
  from pyspark.mllib.evaluation import MulticlassMetrics
  # Keras / Deep Learning
  import tensorflow as tf
  from tensorflowonspark import TFParallel
  from tensorflowonspark import TFCluster
  import argparse
  from keras.models import Sequential
  from keras.layers.core import Dense, Dropout, Activation
  from keras import optimizers, regularizers
  from tensorflow.keras.optimizers import Adam
  #from my model
  import matplotlib
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  import keras
  from tensorflow.keras import layers
  from tensorflow.keras.models import Model
  import pickle
  import shutil
  import random
  import skimage.io as io
  from copy import deepcopy
  import tensorflow_datasets as tfds
  import os
  from functools import reduce
  from google.colab import files
  import os
  import zipfile
  import numpy as np

  sc = SparkContext(conf=SparkConf().setAppName("TFOS"))
  executors = sc._conf.get("spark.executor.instances")
  num_executors = int(executors) if executors is not None else 1

  parser = argparse.ArgumentParser()
  parser.add_argument("--cluster_size", help="number of nodes in the cluster (for S with labelspark Standalone)", type=int, default=num_executors)
  parser.add_argument('--images_labels', type=str, help='Directory for input images with labels')
  parser.add_argument("--export_dir", help="HDFS path to export model", type=str, default="mnist_export")
  parser.add_argument("--output", help="HDFS path to save predictions", type=str, default="predictions")
  args, _ = parser.parse_known_args()
  print("args: {}".format(args))


args: Namespace(cluster_size=1, export_dir='mnist_export', images_labels=None, output='predictions')


In [6]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

Saving kaggle.json to kaggle.json


In [7]:
os.environ["data_source"] = "./Face_Comics_data"

api = KaggleApi()
api.authenticate()

if not os.path.exists(os.environ["data_source"]):
  os.makedirs(os.environ["data_source"])
  if "comic-faces-paired-synthetic-v2" not in os.listdir():
    if "comic-faces-paired-synthetic-v2.zip" not in os.listdir():
      ! kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2
      with zipfile.ZipFile("comic-faces-paired-synthetic-v2.zip", 'r') as f:
        f.extractall("comic-faces-paired-synthetic-v2")
    os.remove("comic-faces-paired-synthetic-v2.zip")


!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/comics" "./Face_Comics_data"
!mv "./comic-faces-paired-synthetic-v2/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/faces" "./Face_Comics_data"

!rm -rf comic-faces-paired-synthetic-v2

100% 2.17G/2.18G [01:15<00:00, 43.7MB/s]
100% 2.18G/2.18G [01:15<00:00, 31.0MB/s]


In [ ]:
#strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [6]:
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

NameError: ignored

In [ ]:
"""
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
"""

In [9]:
data_dir= "./Face_Comics_data"

tf.random.set_seed(123456)

EPOCHS = 15
BATCH_SIZE = 32
#IMG_SIZE = (160, 160)
IMG_SIZE = (350, 350)

os.listdir(data_dir)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="training",
    shuffle=True,
    seed=123456,
    image_size= IMG_SIZE,
    batch_size=BATCH_SIZE)


validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset="validation",
    shuffle=True,
    seed=123456,
    image_size= IMG_SIZE,
    batch_size=BATCH_SIZE)

Found 20000 files belonging to 2 classes.
Using 14000 files for training.
Found 20000 files belonging to 2 classes.
Using 6000 files for validation.


In [11]:
train_label = np.concatenate([y for x, y in train_dataset], axis=0)

test_label = np.concatenate([y for x, y in validation_dataset], axis=0) 

In [15]:
np.shape(test_label)

(6000,)

In [ ]:
#data augmentation

@tf.function
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (180-x)
  else:
    x
  return x

###Randim invert
@tf.function 
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

@tf.function 
class RandomInvert(layers.Layer):
  @tf.function 
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor
  @tf.function 
  def call(self, x):
    return random_invert_img(x)


###Kernel Inizializer Sobel_x
@tf.function 
def hedge_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [-5, -4, 0, 4, 5], 
            [-8, -10, 0, 10, 8], 
            [-10, -20, 0, 20, 10], 
            [-8, -10, 0, 10, 8], 
            [-5, -4, 0, 4, 5]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

@tf.function 
def vertical_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 0, -1], 
            [1, 0, -1], 
            [1, 0, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

@tf.function 
def horizontal_detector(shape, dtype=None):
    print(shape)    
    sobel_x = tf.constant(
        [
            [1, 1, 1], 
            [0, 0, 0], 
            [-1, -1, -1]
        ], dtype=dtype )
    #create the missing dims.
    sobel_x = tf.reshape(sobel_x, (3, 3, 1, 1))

    print(tf.shape(sobel_x))
    #tile the last 2 axis to get the expected dims.
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    return sobel_x

In [16]:
###Create CNN
#Initialize the CNN
cnn = tf.keras.models.Sequential()
"""    
cnn.add(tf.keras.Sequential([
	layers.RandomFlip("horizontal_and_vertical"),
	layers.RandomRotation(0.3),
    layers.RandomContrast(0.5, seed=None),
    RandomInvert(),
    layers.RandomZoom(height_factor=(-0.2, +0.3)),
    layers.RandomTranslation(height_factor=(-0.2, +0.3),width_factor=(-0.2, +0.3)),
    layers.Rescaling(1./255, offset= -1)
    ]))
"""
#DROPOUT
#Introduce the Convolution layer with Kernel_initializer=Sobel_x
#Convolution
cnn.add(tf.keras.layers.Conv2D(32, kernel_size=5, strides=2, activation='relu',  input_shape = [350,350,3]))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Convolution
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,  strides=2, activation = 'relu'))
#Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))
#Second Convolutional Layer
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2,  strides=2, activation = 'relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=1, padding='same'))
#Third Convolutional Layer
#cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 2, activation = 'relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides=2))
#Flattening
cnn.add(tf.keras.layers.Flatten())
#Full Connection
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
#DROPOUTS ok(0.1)
cnn.add(tf.keras.layers.Dropout(0.1))
#Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))
###Training CNN
#Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#training the CNN on the train_dataset and validate it on the validation_dataset
#model_fit = cnn.fit(x = train_dataset, validation_data = validation_dataset, epochs = EPOCHS)

cnn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 173, 173, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 86, 86, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 32)        4128      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 10, 10, 32)       0

In [17]:
tf_classifier = tf.keras.estimator.model_to_estimator(keras_model = cnn)

INFO:tensorflow:Using default config.


2022-05-17 23:10:01,447 INFO (MainThread-63) Using default config.


2022-05-17 23:10:01,455 WARNING (MainThread-63) Using temporary folder as model directory: /tmp/tmpiv5fqh88


INFO:tensorflow:Using the Keras model provided.


2022-05-17 23:10:01,460 INFO (MainThread-63) Using the Keras model provided.
/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpiv5fqh88', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


2022-05-17 23:10:02,246 INFO (MainThread-63) Using config: {'_model_dir': '/tmp/tmpiv5fqh88', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [19]:
input_name = cnn.input_names[0]
input_name

'conv2d_input'

In [23]:
train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
    x = {input_name: train_features},
    y = train_label,
    batch_size = BATCH_SIZE,
    num_epochs = EPOCHS,
    shuffle = True
)

2022-05-17 23:17:03,709 WARNING (MainThread-63) From /usr/local/lib/python3.7/dist-packages/tensorflow/python/util/lazy_loader.py:59: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



NameError: ignored

In [ ]:

  # Running single-node TF instances on each executor
  TFParallel.run(sc, inference, args, args.cluster_size)

  #cluster = TFCluster.run(sc, main_fun, args, args.cluster_size, args.num_ps, args.tensorboard, TFCluster.InputMode.TENSORFLOW)
  #cluster.shutdown()

In [ ]:
#There are two variable objects. Divided between around 2.5 million of MobilNet parameters which 
#are frozen, and 1.2 thousand of trainable parameter in the Dense layer
len(cnn.trainable_variables)

acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']
loss_ = model_fit.history['loss']
val_loss_ = model_fit.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color = 'gray',linestyle='dashed')
plt.plot(val_acc, label='Validation Accuracy', color = 'black')
plt.ylim([0.8, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss_, label='Training Loss', color = 'gray',linestyle='dashed')
plt.plot(val_loss_, label='Validation Loss', color = 'black')
plt.ylim([0, 1.0])
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
